# Ejercicio (resuelto) - Traduccion Automática

En esta sesión veremos una arquitectura secuencia a secuencia (sequence-to-sequence, o *seq2seq*) para traducción neuronal de inglés a español. Implementaremos la red neuronal siguiente una arquitectura encoder-decoder. La estructura se ilustra en la siguiente imagen:

![texto alternativo](https://docs.google.com/uc?id=189LP5y8Q2ocge971-nzUUXQb4yFzDdyr)


Como se puede apreciar, vamos a realizar la traducción **a nivel de caracter**.

## Código Keras paso a paso

Para entrenar un traductor automático neuronal, sólo necesitamos frases en el idioma origen y su correspondiente traducción en el idioma destino. A diferencia de otro tipo de sistemas, no es necesario proporcionar un alineamiento entre los elementos de cada frase.

Para este ejemplo vamos a utilizar el corpus que se proporciona en [este enlace](http://www.dlsi.ua.es/~jcalvo/data/eng-spa.txt). Este corpus se ha extraído de [este repositorio](http://www.manythings.org/anki/), con diversas opciones para otros pares de idiomas.

In [ ]:
import urllib.request

# Número de pares de secuencias que vamos a considerar (aumentalo para un mejor entrenamiento)
limit_of_sentences = 4000

# Se descarga el bitexto (pares inglés-español) y se descompone en lineas
bitext_url = "http://www.dlsi.ua.es/~jcalvo/data/eng-spa.txt"
response = urllib.request.urlopen(bitext_url)
bibtext = response.read().decode('utf-8').splitlines()

# Creamos los corpus de idioma origen y destino
data_input = []
data_output = []

# Creamos los pares de datos limitados por el número de secuencias escogido
for idx in range( min(len(bibtext), limit_of_sentences) ):
    english, spanish = bibtext[idx].split('\t')
    data_input.append( english )
    data_output.append( spanish )

# Comprobación
print('Frase ejemplo inglés: {}'.format(data_input[3]))
print('Frase ejemplo español: {}'.format(data_output[3]))
print()

Los datos vienen ordenados por longitud y frase: la red podría utilizar el orden para *ahorrarse* información en el vector de contexto, lo cual podría ser perjudicial para el aprendizaje. Así pues, barajamos los datos manteniendo el alineamiento con el *ground-truth*

In [ ]:
import random

# Enlazamos ambas listas
common_set = list(zip(data_input, data_output))

# Se barajan y se desenlazan de nuevo
random.shuffle(common_set)
data_input[:], data_output[:] = zip(*common_set)

# Comprobación
print('Frase ejemplo inglés: {}'.format(data_input[3]))
print('Frase ejemplo español: {}'.format(data_output[3]))
print()


En este punto vamos a incorporar los caracteres de comienzo de secuencia (start of sentence, *sos*) y final de secuencia (end of sentence, *eos*). Vamos a utilizar dos caracteres especiales para indicarlos, con atención a que éstos no pertenezcan a los alfabetos de salida.



In [ ]:
# Caracteres de comienzo y final
output_sos = '<'
output_eos = '>'

# Las datos de salida se transforman para incluir ambos
data_output = [output_sos + output_sentence + output_eos for output_sentence in data_output]

# Comprobación
print('Frase ejemplo inglés: {}'.format(data_input[3]))
print('Frase ejemplo español: {}'.format(data_output[3]))
print()

Ahora necesitamos conocer los alfabetos de entrada y salida, especialmente su longitud. Esta información es necesaria para saber la dimension de la representación one hot a utilizar.

In [ ]:
# Conjuntos de caracteres
input_alphabet = set()
output_alphabet = set()

for input_sentence in data_input:
    input_alphabet.update(list(input_sentence))

for output_sentence in data_output:
    output_alphabet.update(list(output_sentence))

# Guardamos sus longitudes en variables
input_alphabet_len = len(input_alphabet)
output_alphabet_len = len(output_alphabet)

# Comprobación
print('Alfabeto de entrada de ' + str(input_alphabet_len) + ' caracteres: ' + str(input_alphabet))
print('Alfabeto de salida de ' + str(output_alphabet_len) + ' caracteres: ' + str(output_alphabet))

Asignamos un valor numérico identificativo a cada caracter, guardando la conversión entre ambas representaciones para el futuro.

In [ ]:
# De caracter a entero
input_alphabet_from_char_to_int = dict([(char, i) for i, char in enumerate(input_alphabet)])
output_alphabet_from_char_to_int = dict([(char, i) for i, char in enumerate(output_alphabet)])

# De entero a caracter
input_alphabet_from_int_to_char = dict([(i, char) for i, char in enumerate(input_alphabet)])
output_alphabet_from_int_to_char = dict([(i, char) for i, char in enumerate(output_alphabet)])

# Comprobación
print('Identificador 0 para el vocabulario de entrada: ' + str(input_alphabet_from_int_to_char[0]))
print('Identificador de \'a\' para el vocabulario de entrada: ' + str(input_alphabet_from_char_to_int['a']))
print('')
print('Identificador 0 para el vocabulario de salida: ' + str(output_alphabet_from_int_to_char[0]))
print('Identificador de \'a\' para el vocabulario de salida: ' + str(output_alphabet_from_char_to_int['a']))

Construimos los generadores que se usarán para alimentar a la red. Debe tenerse en cuenta que hay que hacer *padding* dentro de cada paquete creado, ya que los tensores a devolver deben tener una dimensión definida.

In [ ]:

import numpy as np

# Función auxiliar que convierte listas en paquetes de datos
def batch_to_vectors(data_input, data_output):
  max_input_len = max([len(input_sentence) for input_sentence in data_input])
  max_output_len = max([len(output_sentence) for output_sentence in data_output])

  encoder_input = np.zeros((len(data_input),max_input_len,input_alphabet_len), dtype=np.float)

  for idx_s, input_sentence in enumerate(data_input):
      for idx_c, char in enumerate(input_sentence):
          encoder_input[idx_s][idx_c][input_alphabet_from_char_to_int[char]] = 1.


  decoder_input = np.zeros((len(data_input),max_output_len,output_alphabet_len), dtype=np.float)
  decoder_output = np.zeros((len(data_input),max_output_len,output_alphabet_len), dtype=np.float)

  for idx_s, output_sentence in enumerate(data_output):
      for idx_c, char in enumerate(output_sentence):
          decoder_input[idx_s][idx_c][output_alphabet_from_char_to_int[char]] = 1.
          if idx_c > 0:
              decoder_output[idx_s][idx_c-1][output_alphabet_from_char_to_int[char]] = 1.

  return encoder_input, decoder_input, decoder_output

# Función generadora
def generator(data_input_full, data_output_full, batch_size=16):
  while True:
    for idx in range(0,len(data_input_full), batch_size):
      data_input = data_input_full[idx:idx+batch_size]
      data_output = data_output_full[idx:idx+batch_size]

      x,y,t = batch_to_vectors(data_input,data_output)
      yield [x,y], t


# Inicializamos un generador para la comprobación
check_generator = generator(data_input, data_output)

for _ in range(5):
  [encoder_input, decoder_input], decoder_output = next(check_generator)
  print ('')
  print (encoder_input.shape)
  print (decoder_input.shape)
  print (decoder_output.shape)


Ahora vamos a construir el model neuronal. Cosas a tener en cuenta:
*  ```return_sequences=True``` hace que nos devuelva las predicciones en cada paso.
* ``` return_states=True``` hace que se devuelvan los estados internos.
* Como vamos a utilizar celdas LSTM, los estados internos se componen tanto del estado interno en sí (h) como de la celda memoria (c).

In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

# Encoder
encoder_inputs = Input(shape=(None, input_alphabet_len), name='encoder_input')
encoder_outputs, encoder_h, encoder_c = LSTM(256, return_state=True, name='encoder_output')(encoder_inputs)
context_vector = [encoder_h, encoder_c]

# Decoder
decoder_inputs = Input(shape=(None, output_alphabet_len), name='decoder_input')
decoder_outputs, _, _ = LSTM(256, return_sequences=True, return_state=True, name='decoder_lstm')(decoder_inputs, initial_state=context_vector)
decoder_outputs = Dense(output_alphabet_len, activation='softmax',name='decoder_output')(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()

## Fase de entrenamiento

Vamos a utilizar 1 % del corpus para validar, dejando el 99 % para entrenar la red.

Para la función de entrenamiento, creamos un generador para el entrenamiento y otro para la validación. No obstante, también obtendremos el paquete completo de validación para monitorizar visualmente qué ocurre.

In [ ]:
val_split = 0.01
idx_split = int(len(data_input)*val_split)

data_input_train = data_input[idx_split:]
data_output_train = data_output[idx_split:]

data_input_validation = data_input[:idx_split]
data_output_validation = data_output[:idx_split]

training_generator = generator(data_input_train, data_output_train)
validation_generator = generator(data_input_validation, data_output_validation)

x_val, y_val, t_val = batch_to_vectors(data_input_validation,data_output_validation)

print('Tamaño del conjunto de validación: ' + str(x_val.shape[0]))

Entrenamos con los generadores, para lo cual hay que especificar cuántos pasos corresponden a una época: número de datos dividido por el tamaño de los paquetes de datos. Tras cada época, vamos a comprobar qué resultados de traducción proporciona la red.

In [ ]:
import random

# Entrenamiento durante 50 épocas con super-épocas
for epoch in range(50):
    print('Epoca ' + str(epoch))

    # Entrenamos durante una época
    model.fit_generator(training_generator,
                       steps_per_epoch=len(data_input_train)//16,
                       verbose=2,
                       epochs=1,
                       validation_data=[[x_val,y_val],t_val])

    # Predecimos sobre el paquete de validación
    batch_prediction = model.predict([x_val,y_val],batch_size=16)

    # ---------------------------------------------
    # Comprobamos la traduccion de alguna secuencia
    idx_val_sentence = random.randint(0,x_val.shape[0]-1)
    sentence_prediction = batch_prediction[idx_val_sentence]

    # Obtenemos la frase predicha
    raw_predicted_sequence = [output_alphabet_from_int_to_char[char] for char in np.argmax(sentence_prediction,axis=1)]

    # Leemos solo hasta el caracter EOS
    predicted_sentence = output_sos
    for char in raw_predicted_sequence:
        predicted_sentence += char
        if char == output_eos:
            break

    # Resultado
    print( 'Entrada:\t' + str(data_input_validation[idx_val_sentence]))
    print( 'Predicción:\t' + str(predicted_sentence) )
    print( 'Sal. esperada:\t' + str(data_output_validation[idx_val_sentence]) )
    print('')

### Consideraciones

La traducción entre lenguajes complejos como el inglés y el español es dificil de modelar. Los resultados obtenidos en este ejemplo están lejos de la fiabilidad que alcanzan los esquemas actuales. No obstante, la formulación de los modelos del estado del arte son similares: la diferencia rádica especialmente en la capacidad de cómputo y la cantidad de datos (en este ejemplo es sencillo paliar esto último pero provoca una mayor necesidad de recursos computacionales).

No obstante, existen otras cuestiones que hemos obviado en este ejercicio como la búsqueda de hiper-parámetros adecuados, modelos de lenguaje, (word) embeddings o modelos de atención.